In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from jdt import Jdt

In [ ]:
VIDEO_FILE = './heavy/2021-04-25.mp4'

In [ ]:
class GetFrame:
    # to improve: https://stackoverflow.com/questions/33650974/opencv-python-read-specific-frame-using-videocapture
    def __init__(self):
        self.cache = []
        self.newVideoCapture()
    
    def newVideoCapture(self):
        self.videoCapture = cv.VideoCapture(VIDEO_FILE)
        self.cursor = 0
    
    def next(self):
        ret, frame = self.videoCapture.read()
        self.cursor += 1
        assert ret
        return frame
    
    @lru_cache()
    def __call__(self, frame_i):
        if frame_i < self.cursor:
            self.newVideoCapture()
        with Jdt(frame_i - self.cursor, UPP = 4) as j:
            while self.cursor < frame_i:
                j.acc()
                self.next()
        assert frame_i == self.cursor
        frame = self.next()
        swapped = frame.copy()
        swapped[:,:,0] = frame[:,:,2]
        swapped[:,:,2] = frame[:,:,0]
        return swapped

getFrame = GetFrame()

def widePlot(w = 16, h = 9):
    fig = plt.gcf()
    fig.set_size_inches(w, h)
def view(frame):
    plt.imshow(frame)
    widePlot()
    plt.show()

In [ ]:
def channel(frame, z_keep):
    result = frame.copy()
    for z in range(3):
        if z != z_keep:
            result[:, :, z] = frame[:, :, z_keep] * .5
    return result
def colorize(frame):
    # convert one-channel frame to three-channel
    w, h = frame.shape
    result = np.zeros((w, h, 3), dtype=np.int16)
    ceil  = np.max(frame)
    floor = np.min(frame)
    frame = np.rint((frame - floor) / (ceil - floor) * 255)
    result[:, :, 0] = frame
    result[:, :, 1] = frame
    result[:, :, 2] = frame
    return result

In [ ]:
sample = getFrame(1720)[450:950, :, :]

## 分信道

In [ ]:
view(np.concatenate([sample] + [channel(sample, z) for z in range(3)]))

用蓝色非常合理。  
SM在绿色和红色下几乎不可见。  

## 麦片隐身术

remove cereal body

In [ ]:
view(colorize(sample[:, :, 2] - 2 * (sample[:, :, 1])))

linear system

In [ ]:
x, y = np.linalg.solve(np.array([[-5,11],[26,43]]), np.array([[-22],[-56]]))
x, y

In [ ]:
view(np.concatenate([
    sample, 
    colorize(.65 * sample[:, :, 0] - 1.7 * sample[:, :, 1] + sample[:, :, 2]), 
    channel(sample, 2),
]))